# Swiss Covid-19 Data Analysis

### Import and Basic Config

In [1]:
import altair as alt
alt.data_transformers.enable('data_server')

data_url = 'https://raw.githubusercontent.com/openZH/covid_19/master/COVID19_Fallzahlen_CH_total_v2.csv'

### Helper Functions

In [26]:
def make_line_chart(base, yscale, canton_selection, date_selection, mark_fn):
    line = base.encode(
        y=alt.Y(
            'Anzahl:Q',
            title='Anzahl',
            scale=yscale,
        ),
        color=alt.Color('Kanton:N', scale=alt.Scale(scheme='category20c')),
        opacity=alt.condition(canton_selection, alt.value(1), alt.value(0.1)),
    )
    line = mark_fn(line)

    # Transparent selectors across the chart. This is what tells us
    # the x-value of the cursor
    selectors = base.mark_point().encode(
        opacity=alt.value(0),
    ).add_selection(
        date_selection
    )

    # Draw a rule at the location of the selection
    rules = base.mark_rule(
        color='gray'
    ).encode(
    ).transform_filter(
        date_selection
    )

    total_line = base.mark_line(color='gray').encode(
        y=alt.Y(
            'sum(Anzahl):Q',
            scale=yscale
        ),
        text=alt.Text('sum(Anzahl):Q', format='d'),
    ).transform_filter(
        canton_selection,
    )

    points = total_line.mark_point(
        color='gray'
    ).transform_filter(
        date_selection
    )

    # Text for value on ruler
    points_text = points.mark_text(
        dx=-5,
        dy=-5,
        color='gray',
        align='right',
        fontSize=14,
    )

    # Text for date on ruler
    dates = base.mark_text(
        color='gray',
        dy=-15,
        align='right',
        fontSize=14,

    ).encode(
        y=alt.value(0),
        text=alt.Text('Datum:T', format='%Y-%m-%d'),
    ).transform_filter(
        date_selection
    )

    chart = alt.layer(
        line.add_selection(canton_selection),
        selectors,
        rules,
        total_line,
        points,
        points_text,
        dates,
    ).properties(
        width=1.8*400,
        height=0.8*350
    )
    return chart

def make_table_chart(base, canton_selection, date_selection):
    # Base chart for data tables
    ranked_text = base.mark_text(
        baseline='middle',
    ).encode(
        y=alt.Y('row_number:O', axis=None)
    ).transform_window(
        sort=[{'field': 'Anzahl', 'order': 'descending'}],
        row_number='row_number()',
    ).transform_filter(
        canton_selection
    ).transform_filter(
        date_selection
    )

    # Data Tables
    spalte_kanton = ranked_text.mark_text(
        baseline='middle',
        fontWeight=900,
    ).encode(
        text='Kanton:N',
    ).properties(
        title='Kanton'
    )
    spalte_anzahl = alt.layer(
        ranked_text.mark_rect(
        ).encode(
            color=alt.Color(
                'Anzahl:Q',
                scale=alt.Scale(scheme='viridis'),
                legend=None
            ),
        ),

        ranked_text.encode(
            text=alt.Text('Anzahl:Q', format='d'),
            color=alt.condition(
                'datum.ncantons > 1 ? datum.Anzahl - datum.min_anzahl < 0.8*(datum.max_anzahl-datum.min_anzahl) : true',
                alt.value('white'),
                alt.value('black'),
            ),
        ).transform_window(
            max_anzahl='max(Anzahl)',
            min_anzahl='min(Anzahl)',
            ncantons='count(Kanton)',
            frame=[None, None],
            groupby=['Variable', 'Datum'],
        ),
    ).properties(
        title='Anzahl'
    ).properties(
        width=40
    )
    return spalte_kanton | spalte_anzahl # Combine data tables

In [27]:
%run -n convert_covid_data.py

### Load the Data

In [28]:
df = convert_data(data_url)

### Create the Plot

In [32]:
#variable_dropdown = alt.binding_select(options=sorted(df['Variable'].unique().tolist()), name='Variable  ')
#variable_selection = alt.selection_single(fields=['Variable'], bind=variable_dropdown, init={'Variable': 'Bestätigt Neu'})

selection = alt.selection_multi(fields=['Kanton'], bind='legend')

nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Datum'], empty='none')
base = alt.Chart()
base_chart = base.encode(
    x='Datum:T',
)


lin_chart = make_line_chart(base_chart, alt.Scale(type='linear'), selection, nearest, alt.Chart.mark_area)
log_chart = make_line_chart(base_chart, alt.Scale(type='symlog'), selection, nearest, alt.Chart.mark_line)

tabelle = make_table_chart(base, selection, nearest)

tile = alt.hconcat(
    lin_chart & log_chart,
    tabelle
).resolve_scale(
    x='independent',
    y='independent',
)
#.add_selection(
#    variable_selection
#).transform_filter(
#    variable_selection
#)

chart = alt.vconcat()
for var in df['Variable'].unique():
    chart &= tile.transform_filter(alt.datum.Variable == var).properties(title=var)

### Write the Plot to JSON with a Data URL

In [33]:
chart.data = 'https://raw.githubusercontent.com/wildmichael/swisscovid/main/COVID19_Fallzahlen_CH_total.json'
chart.save('CovidSchweiz.json', format='json')

### Display the Plot with the DataFrame

In [34]:
chart.data = df
chart

alt.VConcatChart(...)